In [1]:
#%%  
"""
                             CWM 2017 for CHE 210A

           Python code to use Psi4 to optimize the water geometry with 
           easy specification of basis set and method

           Uses Psi4 as a python module:   This starting point is written
           to allow the method to be switched easily to rhf, uhf, mp2 or ccsd.

           This script allows the systematic exploration of basis set dependence
           and comparison of rhf and mp2 geometries. 

"""
#%%  
import psi4  # import the psi4 module 
import re as re   # import regular expression library 
import numpy as np   # import numpy for sines and cosines etc. 
#
psi4.set_memory('2048 MB')   # at least 1 gigabyte (1024 MB) for larger basis sets
#%%
#                  BASIS SET AND METHOD CHOICES
#
#       For some of these other options need to be set below 
#       prior to using the psi4.energy() or psi4.optimize() functions
#
basis = "6-311G**"

    # correlation consistent TZ and QZ are available, also 6-31G* etc. See documentation
reference_method = "rhf"  # determines scf method, and source of orbitals for others,
                          #  "uhf" and "rohf" are some  other possibilities
method = "scf"            #  "mp2"  and "ccsd" are other options for which reference = 'rhf' are appropriate
#
print("\n** Starting calculation with basis = ",basis,"  reference = ",
               reference_method, "  method = ",method," **")

#  set up output file labeled by these three choices
#
filestring = 'output_'+reference_method+'_'+method+"_"+"6-311G_double_star"+'.txt'
psi4.core.set_output_file(filestring,False)
print("\n      psi4 output is directed to ",filestring,"\n")
#

#%% 
#                      GEOMETRY
#  Specify the initial guess geometry using cartesian coordinates
#  in angstroms
#
# In the molecule specification we must specify overall charge
#  and spin state for the molecule:
#    0 3   for charge=0 triplet
#    0 1   for singlet 
# Must use "c1" symmetry (no symmetry) for geometry optimizations
#  because the optimization explores any geometry following gradients
#  of potential surface.
#
molecular_geometry  =  """
      0 1 
      O  0.0  0.0  0.0 
      H  0.0  1.42  -1.0
      H  0.0  -1.4  -1.0
    units bohr 
    symmetry c1 """
#
#%%
#
geom_initial =  molecular_geometry
print("initial geometry:\n",geom_initial)
#

psi4.set_options({'basis': basis})
psi4.set_options({'reference': reference_method})
psi4.set_options({'guess': 'gwh'})
psi4.set_options({"scf_type": "df"})  # "direct" also possible, but "df" needed for mp2
psi4.set_options({"MAXITER": 500})
#psi4.set_options({"SOSCF": 'true'})  # second order convergence, slower for larger systems 
molecule_spec = psi4.geometry(geom_initial)
finalenergy, wfcninitial  =psi4.energy(method,molecule=molecule_spec,return_wfn=True) # change to 'scf' for RHF
print("at initial geometry energy is ",finalenergy)
#
#   geometry optimization
#   see Psi4 documentation
#
print("\n ** Begin ",method," geometry optimization **  \n")
finalenergy, wfcn =psi4.optimize(method,molecule=molecule_spec,return_wfn=True )
print("at optimum ",reference_method,"  ",method, "   geometry energy is   ", finalenergy)
#
#  Extract the optimized geometry from the updated molecule class "molecule_spec"
#  to print it out from python and to calculate internal angles and distances
#
#  example of using the method described in the Psi4 manual for analyzing the "molecule" class 
#
geom_matrix=molecule_spec.geometry()
natoms, ncoords =geom_matrix.shape
print("number of atoms = ",natoms)
print("          Hartree-Fock GEOMETRY ")
print("   charge                coordinates")
for n in range(natoms):
    print(n+1," ",molecule_spec.charge(n),geom_matrix.np[n,:])
#
moltestcharge=molecule_spec.molecular_charge()
print("overall molecular charge = ",moltestcharge)
print("\n")
#
#  Use this information to compute bond distances and angles
#  geom_matrix.np[0,i] is the first atoms coordinates i = 1,2,3 
#
print("\n  Optimum OH distances and HOH angle-- ")
O_1 = [ geom_matrix.np[0,0], geom_matrix.np[0,1], geom_matrix.np[0,2] ]
H_1 = [ geom_matrix.np[1,0], geom_matrix.np[1,1], geom_matrix.np[1,2] ]
H_2 = [ geom_matrix.np[2,0], geom_matrix.np[2,1], geom_matrix.np[2,2] ]
# calculate OH distances    
OH_1 = np.subtract(O_1,H_1)
dist1 = np.sqrt(np.dot(OH_1,OH_1))
print("   OH distance = ",dist1)
OH_2 = np.subtract(H_2,O_1)
dist2 = np.sqrt(np.dot(OH_2,OH_2))
print("   OH distance = ",dist2)
#  calculate HOH angles     
cosang = -np.dot(OH_1,OH_2)/(dist1*dist2)
angle = np.arccos(cosang)*180./np.pi
print( " HOH bond angle = ", angle)
# %%

exit()


** Starting calculation with basis =  6-311G**   reference =  rhf   method =  scf  **

      psi4 output is directed to  output_rhf_scf_6-311G_double_star.txt 

initial geometry:
 
      0 1 
      O  0.0  0.0  0.0 
      H  0.0  1.42  -1.0
      H  0.0  -1.4  -1.0
    units bohr 
    symmetry c1 
at initial geometry energy is  -76.04513011676845

 ** Begin  scf  geometry optimization **  

Optimizer: Optimization complete!
at optimum  rhf    scf    geometry energy is    -76.04700945061909
number of atoms =  3
          Hartree-Fock GEOMETRY 
   charge                coordinates
1   8.0 [ 0.00018207 -0.12050663  0.        ]
2   1.0 [-1.41650891  0.9541437   0.        ]
3   1.0 [1.41361928 0.95838392 0.        ]
overall molecular charge =  0



  Optimum OH distances and HOH angle-- 
   OH distance =  1.7781694702479385
   OH distance =  1.7781477910956718
 HOH bond angle =  105.46251604150288


In [14]:
exit()

In [1]:
quit()